In [1]:
import pandas as pd
import numpy as np
import requests
import json
from config import api_key
import flr_tools

In [2]:
start_date = "2014-09-01"
end_date = "2018-02-04"
#query_flr_url =f"https://api.nasa.gov/DONKI/FLR?startDate={start_date}&endDate={end_date}&api_key={api_key}"

In [3]:
dont_use_nasa_time_format = False
flare_alldf = flr_tools.flare_query_all(start_date, end_date, dont_use_nasa_time_format)
flare_alldf

,flare_id,start_time(zulu),peak_time(zulu),end_time(zulu),class_type,power(w/m^2),linked_events
0,2016-11-29T17:19:00-FLR-001,"11/29/2016, 17:19:00","11/29/2016, 17:23:00","11/29/2016, 17:26:00",M1.0,1.000000e-05,[None Found.]
1,2016-11-29T23:29:00-FLR-001,"11/29/2016, 23:29:00","11/29/2016, 23:38:00","11/30/2016, 23:40:00",M1.2,1.200000e-05,[None Found.]
2,2016-12-10T16:48:00-FLR-001,"12/10/2016, 16:48:00","12/10/2016, 17:15:00","12/10/2016, 17:35:00",C4.0,4.000000e-06,[{'activityID': '2016-12-10T17:24:00-CME-001'}]
3,2017-01-21T07:23:00-FLR-001,"01/21/2017, 07:23:00","01/21/2017, 07:26:00","01/21/2017, 07:37:00",C9.3,9.300000e-06,[None Found.]
4,2017-03-27T11:07:00-FLR-001,"03/27/2017, 11:07:00","03/27/2017, 11:12:00","03/27/2017, 12:43:00",C3.2,3.200000e-06,[{'activityID': '2017-03-27T12:12:00-CME-001'}]
...,...,...,...,...,...,...,...
66,2017-09-20T19:28:00-FLR-001,"09/20/2017, 19:28:00","09/20/2017, 19:32:00","09/20/2017, 19:49:00",B8.4,8.400000e-07,[None Found.]
67,2017-09-26T02:31:00-FLR-001,"09/26/2017, 02:31:00","09/26/2017, 02:34:00","09/26/2017, 02:43:00",C1.8,1.800000e-06,[None Found.]
68,2017-09-27T00:49:00-FLR-001,"09/27/2017, 00:49:00","09/27/2017, 01:00:00","09/27/2017, 01:16:00",C1.7,1.700000e-06,[None Found.]
69,2017-10-20T23:10:00-FLR-001,"10/20/2017, 23:10:00","10/20/2017, 23:28:00","10/21/2017, 23:37:00",M1.1,1.100000e-05,[{'activityID': '2017-10-21T00:12:00-CME-001'}]


In [20]:
cme_id = []
for event in flare_alldf['linked_events'][4]:
    cme_id.append(event)

mars_search = []
[mars_search.append(flr_tools.cme_compare(id,start_date,end_date)) for id in cme_id]
mars_search     

[[{'Connection': 'Event not found.', 'Arrival Time': 'No data'}]]

In [4]:
use_nasa_time = True
flare_smalldf = flr_tools.flare_query_small(start_date, end_date, use_nasa_time)
flare_smalldf

,flare_id,peak_time(zulu),class_type,power(w/m^2)
0,2017-01-21T07:23:00-FLR-001,2017-01-21T07:26Z,C9.3,0.000009
1,2017-03-27T11:07:00-FLR-001,2017-03-27T11:12Z,C3.2,0.000003
2,2017-03-27T17:55:00-FLR-001,2017-03-27T18:20Z,C5.1,0.000005
3,2017-04-01T19:30:00-FLR-001,2017-04-01T19:56Z,C3.7,0.000004
4,2017-04-01T21:35:00-FLR-001,2017-04-01T21:48Z,M4.4,0.000044
5,2017-04-02T02:43:00-FLR-001,2017-04-02T02:46Z,C8.0,0.000008
6,2017-04-02T07:48:00-FLR-001,2017-04-02T08:02Z,M5.3,0.000053
7,2017-04-02T12:54:00-FLR-001,2017-04-02T13:00Z,M2.3,0.000023
8,2017-04-02T18:18:00-FLR-001,2017-04-02T18:38Z,M2.1,0.000021
9,2017-04-02T20:28:00-FLR-001,2017-04-02T20:33Z,M5.7,0.000057


In [3]:
query_cme_url =f"https://kauai.ccmc.gsfc.nasa.gov/DONKI/WS/get/CME?startDate={start_date}&endDate={end_date}"
cme_request = requests.get(query_cme_url).json()

In [4]:
cme_request

[{'activityID': '2016-09-02T06:18:00-CME-001',
  'startTime': '2016-09-02T06:18Z',
  'sourceLocation': '',
  'activeRegionNum': None,
  'instruments': [{'id': 2, 'displayName': 'SOHO: LASCO/C3'},
   {'id': 1, 'displayName': 'SOHO: LASCO/C2'},
   {'id': 5, 'displayName': 'STEREO A: SECCHI/COR2'}],
  'cmeAnalyses': [{'time21_5': '2016-09-02T19:15Z',
    'latitude': 35.0,
    'longitude': -87.0,
    'halfAngle': 27.0,
    'speed': 218.0,
    'type': 'S',
    'isMostAccurate': True,
    'note': '',
    'levelOfData': 0,
    'enlilList': None}],
  'linkedEvents': None,
  'note': 'Start time is time when seen in SOHO LASCO C3 due to a data gap in SOHO LASCO C2 imagery.',
  'catalog': 'SWRC_CATALOG'},
 {'activityID': '2016-09-03T12:12:00-CME-001',
  'startTime': '2016-09-03T12:12Z',
  'sourceLocation': '',
  'activeRegionNum': None,
  'instruments': [{'id': 1, 'displayName': 'SOHO: LASCO/C2'},
   {'id': 2, 'displayName': 'SOHO: LASCO/C3'},
   {'id': 5, 'displayName': 'STEREO A: SECCHI/COR2'}]

In [5]:
cme_analysis_data = flr_tools.cme_analysis_df(cme_request)
cme_analysis_data

,activity_id,speed,data_level,lat,long,type,half_angle,is_most_accurate,time21_5,note
0,2016-09-02T06:18:00-CME-001,218,0,35,-87,S,27,True,2016-09-02T19:15Z,
1,2016-09-03T12:12:00-CME-001,108,0,6,114,S,23,True,2016-09-04T18:50Z,Measurement done in SWPC cat using SOHO C2 and...
2,2016-09-04T13:25:00-CME-001,216,0,-14,26,S,32,True,2016-09-05T08:25Z,
3,2016-09-06T06:24:00-CME-001,539,0,22,160,C,10,True,2016-09-06T13:11Z,Measured using C3 and STA.
4,2016-09-06T08:54:00-CME-001,674,1,-20,120,C,31,True,2016-09-06T14:18Z,
...,...,...,...,...,...,...,...,...,...,...
318,2017-12-29T13:25:00-CME-001,303,0,19,129,S,19,True,2017-12-30T00:46Z,
319,2018-01-01T00:39:00-CME-001,279,0,21,112,S,17,True,2018-01-01T18:47Z,
320,2018-01-22T03:12:00-CME-001,560,1,-18,66,C,31,True,2018-01-22T09:28Z,The CME had an irregular form and could only b...
321,2018-01-28T18:24:00-CME-001,241,0,28,123,S,16,True,2018-01-29T09:12Z,


In [3]:
flare_cme_combo_test = flr_tools.cme_compare(start_date, end_date)

In [4]:
flare_cme_combo_test

,activity_id,speed,data_level,lat,long,type,half_angle,is_most_accurate,time21_5,note,flare_id,start_time(zulu),peak_time(zulu),end_time(zulu),class_type,power(w/m^2),linked_events
0,2014-09-03T14:12:00-CME-001,606,0,-9,-111,C,14,True,2014-09-03T20:05Z,using STEREO_cat with C3 and STA,2014-09-03T13:20:00-FLR-001,"09/03/2014, 13:20:00","09/03/2014, 13:53:00","09/03/2014, 13:54:00",M2.5,0.000025,2014-09-03T14:12:00-CME-001
1,2014-09-09T00:16:00-CME-001,780,0,26,-30,C,43,True,2014-09-09T04:16Z,Only LASCO images were available; SWPC_Cat pro...,2014-09-08T23:12:00-FLR-001,"09/08/2014, 23:12:00","09/09/2014, 00:28:00","09/09/2014, 01:30:00",M4.6,0.000046,2014-09-09T00:16:00-CME-001
2,2014-09-10T18:18:00-CME-001,1400,0,15,10,O,45,True,2014-09-10T20:00Z,Only LASCO C3 imagery available. Ambiguity in ...,2014-09-10T17:21:00-FLR-001,"09/10/2014, 17:21:00","09/10/2014, 17:45:00","09/10/2014, 18:20:00",X1.6,0.000160,2014-09-10T18:18:00-CME-001
3,2014-09-14T03:12:00-CME-001,430,0,7,32,S,45,True,2014-09-14T12:00Z,Updated analysis based on all 3 viewing angles.,2014-09-14T02:03:00-FLR-001,"09/14/2014, 02:03:00","09/14/2014, 02:16:00","09/14/2014, 02:36:00",M1.5,0.000015,2014-09-14T03:12:00-CME-001
4,2014-09-24T00:24:00-CME-001,348,0,1,-69,S,27,True,2014-09-24T11:55Z,,2014-09-23T23:02:00-FLR-001,"09/23/2014, 23:02:00","09/23/2014, 23:15:00","09/23/2014, 23:28:00",M2.3,0.000023,2014-09-24T00:24:00-CME-001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,2017-09-08T04:17:00-CME-001,442,0,-5,53,S,21,True,2017-09-08T05:29Z,,2017-09-08T03:39:00-FLR-001,"09/08/2017, 03:39:00","09/08/2017, 03:43:00","09/08/2017, 03:46:00",M1.2,0.000012,2017-09-08T04:17:00-CME-001
97,2017-09-08T07:24:00-CME-001,450,0,-3,54,S,20,True,2017-09-08T14:03Z,,2017-09-08T07:40:00-FLR-001,"09/08/2017, 07:40:00","09/08/2017, 07:49:00","09/08/2017, 07:58:00",M8.1,0.000081,2017-09-08T07:24:00-CME-001
98,2017-09-10T16:09:00-CME-001,2650,1,-12,85,R,54,True,2017-09-10T17:18Z,,2017-09-10T15:35:00-FLR-001,"09/10/2017, 15:35:00","09/10/2017, 16:06:00","09/10/2017, 16:31:00",X8.2,0.000820,2017-09-10T16:09:00-CME-001
99,2017-09-12T08:09:00-CME-001,450,0,-9,-36,S,22,True,2017-09-12T16:30Z,Measure STEREO_CAT to velocities <400 km/s.,2017-09-12T07:23:00-FLR-001,"09/12/2017, 07:23:00","09/12/2017, 07:29:00","09/12/2017, 07:48:00",C3.0,0.000003,2017-09-12T08:09:00-CME-001


In [6]:
len(flare_cme_combo_test)

305545